In [2]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
from connection_keys import connect_bq_client
import statsmodels.api as sm
import os
import warnings
import itertools
warnings.filterwarnings("ignore")



ImportError: cannot import name 'bigquery' from 'google.cloud' (unknown location)

In [3]:
import os

GCP_PROJECT_ID = os.environ['GCP_PROJECT_ID']
GCP_SERVICE_ACCOUNT_FILE = os.environ['GCP_SERVICE_ACCOUNT_FILE']

KeyError: 'GCP_PROJECT_ID'

In [ ]:
GCP_PROJECT_ID = os.environ['GCP_PROJECT_ID']
GCP_SERVICE_ACCOUNT_FILE = os.environ['GCP_SERVICE_ACCOUNT_FILE']
GCP_TABLE = os.environ['GCP_TABLE']